In [1]:
# saves you having to use print as all exposed variables are printed in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# suppress warning message
# import warnings
# warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import torch
import pyarrow as pa
import pyarrow.compute as pc

---------

In [4]:
from nn_rag import Knowledge
from nn_rag.components.commons import Commons

In [5]:
# create instance of capability
kn = Knowledge.from_memory()

In [6]:
tbl = kn.set_source_uri('hadron/source/Global-Index-1st-Edition-Report.pdf').load_source_canonical()

In [27]:
doc = pymupdf.open('hadron/source/Global-Index-1st-Edition-Report.pdf')

In [37]:
for page in doc:
    tabs = page.find_tables()
    if tabs.tables:
        for t in tabs:
            print(t.extract())
            print(t.header.cells)
            
        break


[None, 'OUNTRY REG', 'ION', 'INDEX\nSCORE', 'pillar score', None, None, None, None]
[None, (115.40550102377837, 208.0, 207.25309368896484, 251.4), (207.25309368896484, 208.0, 245.03703703703704, 251.4), (245.03703703703704, 208.0, 286.3909090909091, 251.4), (286.3909090909091, 208.0, 412.04464285714283, 227.8041194209346), None, None, None, None]
